We will assume the following in this problem: 

You have 100 users, who use your main app, which resembles Spotify. 
You are launching a new feature for an app imitating “Discover weekly”. 
This new feature will also suggest to the users what songs they can listen to, that they may like.
This means that you will be looking at what songs people played in the app and we will suggest new songs to them,
in the new feature, based on what we know. The suggested 5 songs may be songs the user has played. 


Week 1 after the day of the launch - Individual choice
Each of your 100 users is trying the new Discover feature in your app for the first time, in the first week. The feature will generate songs that he or she may like. You need to generate 5 songs. Assume this is the first week in which you launched your app on the google store. 
 
Assume that all users start using the feature on day 1 and no one downloads it later. The 5 songs that will be generated for the first time we pick in the following way:  
 
You already have 100 pre-made playlists, with 50 songs in each. Your task is the following: if you find a playlist that contains 3 songs that the user has listened to already and 3 he has not heard yet, you select this playlist. From this playlist, your algorithm picks any 5 songs. 


Task
Write a Python computer program that can simulate Spotify’s “Discover Weekly” as mentioned above.

Minimum requirements

- Give the pseudocode of your program.

- Make a flowchart of your program.

- Put comments in your code.

- The program generates suggestions for the users of Spotify: a few new songs every few days.

 

In [2]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
import random
import array
import time
import numpy

from numpy import arange, average, exp, array, dot
from numpy import ndarray
from collections import Counter

In [ ]:
df = pd.read_csv('spotify-dataset.csv')
df_playlists = pd.DataFrame() # Create a new dataframe to store the playlists
#df
playlists = {}
for i in range(0,100): # Loop through the first 100 playlists
    df2 = []
    for j in range(0,50): # Loop through the first 50 songs in each playlist
        df2.append(df.sample().values.tolist()[0]) # Get a random song from the dataset
    playlists[i] = df2
print(playlists)

In [4]:
# Creating basic Dataframes that will be used for the remaining tasks
df.columns = [c.replace(' ', '_') for c in df.columns]
df_train = pd.DataFrame() #Used in task 3
#print(df.columns[-9:])



def normalize(df): # Normalizing dataframe for better results in task 3
    result = df.copy()
    for feature_name in df.columns[-9:]:
        max_value = df[feature_name].astype(float).max()
        min_value = df[feature_name].astype(float).min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result
df_norm=normalize(df)

In [5]:


#Creating a user class where we can define a liked_songs list(for the users preferances) and a categories list to categorize what the user listens to(Task 3)
class User: 
    liked_songs = []
    categories = []
    def __init__(self) -> None:
        self.liked_songs = []
        self.categories = []
        
    def add_random_song(self): # Adding songs to the liked_songs list
        self.liked_songs.append(df.sample().values.tolist()[0]) # Get a random song from the dataset
    
    def add_random_song_norm(self): #This method adds a normalized song to the users liked songs ( USED ONLY IN TASK 3....ONLY FOR BETTER RESULTS WITH JACCARD SIMILARITY)
        self.liked_songs.append(df_norm.sample().values.tolist()[0])


    def find_playlist(self,playlists): 
        common_songs =[]
        for i in playlists:
            playlist = (playlists[i]) 
            #print(playlist)
            playlistUpdated = []
            for song in playlist:
               # print(song[0])
                playlistUpdated.append(song[0])
            playlist = playlistUpdated
            #print(playlist)
            counter_exists =0 # counter for the same songs in both playlists
            counter_no_exist = 0 # counter for the songs that are not in the playlist

            for song in self.liked_songs: # Loop through the songs the user listened to
                if song[0] in playlist:
                    counter_exists+=1
                    common_songs.append(song)
                else:
                    counter_no_exist+=1 
            if counter_exists >2 and counter_no_exist >2: 
                return playlist,common_songs
    
    def determine_category(self):
        genres ={}
        for songs in self.liked_songs:
            #print(songs[2])
            genre = songs[2]
            if 'pop' in genre:
                genres['pop'] = genres.get('pop',0) + 1
            elif 'rock' in genre:
                genres['rock'] = genres.get('rock',0) + 1
            elif 'techno' in genre:
                genres['techno'] = genres.get('techno',0) + 1
        maximum = 0
        max2 = 0
        max_category = ""
        max2cat = ""
        for i in genres:
            if genres[i] >maximum:
                maximum =genres[i]
                max_category = i
            if genres[i]<maximum and genres[i]>max2:
                max2 = genres[i]
                max2cat = i
        
        if max2 != 0:
            if int((maximum/max2)*10) in range(9,10):
                return [max_category,max2cat]
            #print(max_category)
        return [max_category]


            
#Task 1: Creating user
user1 = User()
for i in range(0,50):
    user1.add_random_song() # Add 50 random songs to the user's listened songs



selected_playlist = user1.find_playlist(playlists)[0]

selected_songs =random.sample(selected_playlist,5)

print(selected_songs)


['A L I E N S', 'Kissing Strangers', "How Ya Doin'? (feat. Missy Elliott)", 'Here', 'Everybody Talks']


Week 2 - Genres
In the second week you will use a different approach: 
A user opens the app and is looking for new suggestions on “Discover.”  
 
There are three types of music only: pop, rock, and techno.  
 
You want to find which style the user has listened to the most, of those three. Define in your own terms what this should mean. For example, you can say that if the user has listened to 20 rock songs and 2 techno songs, and 3 pop songs, this is a rock music fan. Or you can work with percentages: you can find a user that played 80% techno and 20% pop. You can make your algorithm disregard a difference of 10% and say that if a user played pop 45 percent of the time and rock 50 percent of the time, the difference is small and both styles should be considered equally important to him or her. Outline your scale and assumptions in your work. 

In the second week, your algorithm should suggest 5 songs again, this time based on the above-mentioned, i.e. based on style. 
 
It is possible that the program suggests one song in one week and again the same song in the following week.

In [6]:
user2 = User()
for i in range(0,50):
    user2.add_random_song() # Add 50 random songs to the user's listened songs

#print(user2.liked_songs) # Sample category from song

user2.determine_category()
df3 = pd.DataFrame()


def rec_song(): #recommends songs
    df3 = df[df.the_genre_of_the_track == user2.determine_category()[0]].sample(5)
    return df3
rec_song()



,title,artist,the_genre_of_the_track,year,Beats.Per.Minute_-The_tempo_of_the_song,"Energy-_The_energy_of_a_song_-_the_higher_the_value,_the_more_energtic","Danceability_-_The_higher_the_value,_the_easier_it_is_to_dance_to_this_song","Loudness/dB_-_The_higher_the_value,_the_louder_the_song","Liveness_-_The_higher_the_value,_the_more_likely_the_song_is_a_live_recording","Valence_-_The_higher_the_value,_the_more_positive_mood_for_the_song",Length_-_The_duration_of_the_song,Acousticness_-_The_higher_the_value_the_more_acoustic_the_song_is,Speechiness_-_The_higher_the_value_the_more_spoken_word_the_song_contains,Popularity-_The_higher_the_value_the_more_popular_the_song_is
510,I Like It,Cardi B,pop,2018,136,73,82,-4,37,65,253,10,13,85
64,Marry You,Bruno Mars,pop,2011,145,83,62,-5,10,48,230,33,4,73
227,Love Never Felt So Good,Michael Jackson,pop,2014,118,72,78,-6,7,71,246,13,4,71
407,Out Of The Woods,Taylor Swift,pop,2016,92,84,55,-7,34,34,236,0,4,66
107,Payphone,Maroon 5,pop,2012,110,75,74,-5,29,55,231,2,4,79


Week 3 - Shifts
In the third week after the launch, you want to account for a user’s mood shift. We have a few types of songs: “happy”, “party”, “calming”, and “lounge.” Define in your own algorithm if you think one song can be classified as two of those at the same time.  
 
This week you need to select 5 more songs to suggest to each user, again. If last week the user was playing a lot of songs from one of those types, this week we want to suggest more of the same type. Define, in your own terms what this should mean: if the user listened to 3 “calming” songs, should we provide 3 more without considering other factors? What if he or she also listened to 3 “party” songs and 4 “lounge” style songs?

In [7]:

import random # Re-importing libraries because random exists in numpy too but we want the random library(I will fix this later on by changing the names of the imported libraries)


training_set_inputs = [] #Creating basic list for training NB and NN
training_set_outputs = [[]] # Outputs list for classification

N = 10 # Number of songs to categorize
for i in random.sample(range(0,len(df)), N): # Manually Categorizing N amount of songs for the NB and the NN to use as training sets
    training_set_inputs.append(df.loc[i, :].values.tolist()[-9:])
    print(df.loc[i, :].values.tolist())
    time.sleep(1) # only used because sometimes the print output is not shown so the user does not know the name of the songs he is categorizing
    training_set_outputs[0].append(input("Please enter a category: #happy = 1 #party = 2 #calming =3# lounge = 4"))


#print(training_set_inputs)

tr_in = training_set_inputs #Saving these as lists just to have them as both list data types and array data types
tr_out = training_set_outputs[0]


df_train['X'] = training_set_inputs #Addding to data frame just in case they are needed
df_train['Y'] = training_set_outputs[0]


training_set_outputs = numpy.array(training_set_outputs).T # Turning the training lists into arrays for NN input
training_set_inputs = numpy.array(training_set_inputs)

['Stay', 'Zedd', 'complextro', 2017, 102, 62, 69, -5, 12, 54, 210, 25, 6, 80]
["Really Don't Care", 'Demi Lovato', 'dance pop', 2015, 121, 73, 71, -4, 19, 85, 202, 12, 9, 70]
['Rise', 'Katy Perry', 'dance pop', 2016, 101, 64, 55, -6, 11, 44, 203, 5, 5, 66]
['Trampoline (with ZAYN)', 'SHAED', 'electropop', 2019, 127, 46, 62, -6, 14, 50, 184, 56, 3, 92]
['Dusk Till Dawn - Radio Edit', 'ZAYN', 'dance pop', 2018, 180, 44, 26, -7, 11, 10, 239, 10, 4, 83]
['Cool Girl', 'Tove Lo', 'dance pop', 2016, 102, 69, 67, -7, 8, 20, 199, 31, 11, 69]
['Wait', 'Maroon 5', 'pop', 2018, 126, 60, 66, -5, 11, 45, 191, 10, 6, 67]
['A Sky Full of Stars', 'Coldplay', 'permanent wave', 2014, 125, 68, 55, -6, 21, 16, 268, 1, 3, 60]
['Jealous - Remix', 'Nick Jonas', 'dance pop', 2015, 93, 71, 68, -4, 44, 45, 222, 1, 8, 70]
['Tell Me You Love Me - NOTD Remix', 'Demi Lovato', 'dance pop', 2018, 102, 77, 58, -5, 9, 76, 182, 5, 8, 44]


In [8]:
sample_test = df.loc[555, :].values.tolist()[-9:] # Taking a sample song that we will check predicted category of....saving only the colums that are used to categorize the song
print("Sample Test")
print(df.loc[555, :].values.tolist())


Sample Test
['Wait', 'Maroon 5', 'pop', 2018, 126, 60, 66, -5, 11, 45, 191, 10, 6, 67]


In [9]:
#Nueral network class
from numpy import random
class NeuralNetwork():
    def __init__(self):
        random.seed(1)
        #Size of matrix has to do with inputs(Currently using 9 columns for classification)
        self.synaptic_weights = 2 * random.random((9, 1)) - 1

# I wont go into detail about what all these functions do but they are the "guts" of the nueral network
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))


    def __sigmoid_derivative(self, x):
        return x * (1 - x)


    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in range(number_of_training_iterations):
            output = self.think(training_set_inputs)
            error = training_set_outputs - output
            adjustment = dot(training_set_inputs.T, error * self.__sigmoid_derivative(output))
            self.synaptic_weights += adjustment

  
    def think(self, inputs):
        return self.__sigmoid(dot(inputs, self.synaptic_weights))


In [10]:

from numpy import random

#Create NB Model and Nueral Network Instance
model = GaussianNB()
neural_network = NeuralNetwork()

#Fit the data into the NB Model
model.fit(tr_in, tr_out)


GaussianNB()

In [11]:


import collections #Importing this library just to find frequency of predicted categories ( to be used to classify which categories the user mostly listens too)

#Creating 3rd user for task 3
user3 = User()
for i in range(0,50):
    user3.add_random_song_norm() #This time adding the normalized songs to his liked songs

df_nueral = pd.DataFrame(user3.liked_songs)

#Predicting the category that the user listens to (using NB model)
def predict_categories(liked_songs):
    training_set_a = []
    for song in liked_songs:
        training_set_a.append(song[-9:])
    
    predicted_categories = []
    for song in training_set_a:
        predicted_categories.append(ndarray.tolist(model.predict([song]))[0])
    return predicted_categories


user3.categories = predict_categories(user3.liked_songs) #Adding the categories to the list

# Figuring out which category is the most common
Counter = collections.Counter(user3.categories)
chosen_category = [max(set(user3.categories), key = user3.categories.count)]
most_occur = Counter.most_common(4)
print(most_occur)

#Checkling if the difference between the two categories is less than 10 %
if(len(most_occur)>1):
    print(100-(most_occur[1][1] / most_occur[0][1]) * 100)
    if 100-(most_occur[1][1] / most_occur[0][1]) * 100 <10:
        chosen_category = [most_occur[0][0] , most_occur[1][0]]
print(chosen_category) # Print the categories that the user relates to



[('1', 50)]
['1']


In [12]:
from sklearn.cluster import KMeans


#Testing out kmeans clustering for classification
kmeans = KMeans(n_clusters=4, random_state=0).fit(tr_in)

#Adding the combined categories together so that they can be added as parameters for the classification techniques
df_norm["Raw Text"] = ""

#Here we add all the columns of the data we want to use into one as a list, this is because our classification techniques require lists as sample inputs
for cols in df.columns[4:13]:
    df_norm["Raw Text"] +=  df_norm[cols].astype(str) +"," 

#All of this this just preprocessing the data and making sure it applies to our datatype needs (Aka being floats instead of strings)
df_norm["Raw Text"] = df_norm["Raw Text"].apply(lambda x: x.split(","))
df_norm["Raw Text"] = df_norm["Raw Text"].apply(lambda x: x[0:9])
df_norm["Raw Text"] = df_norm["Raw Text"].apply(lambda x: [float(y) for y in x if y !=''])
df_norm["Raw Text"] = df_norm["Raw Text"].apply(lambda x: [float(y) for y in x])
df_norm["Raw Text"] = df_norm["Raw Text"].apply(lambda x: [x])

#Add columns to the dataframe for each predictions technique for each song in the datafram
df['Predicted Category NB'] = df_norm["Raw Text"].apply(model.predict)
df['Predicted Category NN'] = df_norm["Raw Text"].apply(neural_network.think)
df['Predicted Category Clustering KM'] = df_norm['Raw Text'].apply(kmeans.predict)

df #Print dataframe to make sure it looks good


,title,artist,the_genre_of_the_track,year,Beats.Per.Minute_-The_tempo_of_the_song,"Energy-_The_energy_of_a_song_-_the_higher_the_value,_the_more_energtic","Danceability_-_The_higher_the_value,_the_easier_it_is_to_dance_to_this_song","Loudness/dB_-_The_higher_the_value,_the_louder_the_song","Liveness_-_The_higher_the_value,_the_more_likely_the_song_is_a_live_recording","Valence_-_The_higher_the_value,_the_more_positive_mood_for_the_song",Length_-_The_duration_of_the_song,Acousticness_-_The_higher_the_value_the_more_acoustic_the_song_is,Speechiness_-_The_higher_the_value_the_more_spoken_word_the_song_contains,Popularity-_The_higher_the_value_the_more_popular_the_song_is,Predicted Category NB,Predicted Category NN,Predicted Category Clustering KM
0,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83,[1],[[1.9209958511883074e-08]],[3]
1,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82,[1],[[5.876333151939032e-08]],[3]
2,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80,[1],[[3.370974732694814e-10]],[3]
3,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79,[1],[[4.734786172060296e-10]],[3]
4,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78,[1],[[3.750448426664065e-09]],[3]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,Find U Again (feat. Camila Cabello),Mark Ronson,dance pop,2019,104,66,61,-7,20,16,176,1,3,75,[1],[[1.0377841666988455e-08]],[3]
599,Cross Me (feat. Chance the Rapper & PnB Rock),Ed Sheeran,pop,2019,95,79,75,-6,7,61,206,21,12,75,[1],[[2.7780200303985344e-08]],[3]
600,"No Brainer (feat. Justin Bieber, Chance the Ra...",DJ Khaled,dance pop,2019,136,76,53,-5,9,65,260,7,34,70,[1],[[3.039160155476451e-11]],[3]
601,Nothing Breaks Like a Heart (feat. Miley Cyrus),Mark Ronson,dance pop,2019,114,79,60,-6,42,24,217,1,7,69,[1],[[1.4326777036431302e-09]],[3]


Overall after taking a better look at the data we came to the conclusion that Neural Networks, Kmeans clusstering or Naive Bayes are all not valid approaches for this problem. These techniques typically need structured and labeled data for them to give valuable predictions, the kind of data we unfortunaltey do not have. However, we attempted to create that data by personally classifying songs based on our preferances. This turned out to be tedious work and we always fell short on the amount of data we needed. After some reasearch, we found one approach that we believe is the best for this sort of problem. Commonly known as Collaborative Filtering, this technique is actually used in most reccomendation engines. We decided that we would try it out. From the two main approaches, User-based collaborative filtering and item-based collaborative filtering, we opted to go with item-based. The code below is what is used in our final evaluation and solution for task 3. Enjoy:)

In [13]:

#Experimentation with Jaccard Similarity for Reccomendations

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def jaccard_set(list1, list2): #Computing similarity between two lists (These two lists are going to be a song with the liked songs of the user)
    """Define Jaccard Similarity function for two sets"""
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union



# Here we get the similarity between a song and the liked songs of the user 
def get_jaccard_similarity(sample_song : list):
    similarity_score =0
    for song in user3.liked_songs:
        similarity_score += jaccard_set(sample_song,song[4:13])
    return similarity_score/len(user3.liked_songs) # Our similarity score is the average similarity of each all the songs  


df['Jaccard Similarity'] = df_norm['Raw Text'].apply(lambda x: get_jaccard_similarity(x[0])) # Apply the similarity technique for each song to compute how similar it is to the users song taste


#Sort the dataframe to get the top 5 songs (the most similar songs) in the first 5 rows
dfs = df.sort_values(['Jaccard Similarity'], ascending=[False])
dfs.head(5) #Print out the songs that are reccomended based on the users liked songs

,title,artist,the_genre_of_the_track,year,Beats.Per.Minute_-The_tempo_of_the_song,"Energy-_The_energy_of_a_song_-_the_higher_the_value,_the_more_energtic","Danceability_-_The_higher_the_value,_the_easier_it_is_to_dance_to_this_song","Loudness/dB_-_The_higher_the_value,_the_louder_the_song","Liveness_-_The_higher_the_value,_the_more_likely_the_song_is_a_live_recording","Valence_-_The_higher_the_value,_the_more_positive_mood_for_the_song",Length_-_The_duration_of_the_song,Acousticness_-_The_higher_the_value_the_more_acoustic_the_song_is,Speechiness_-_The_higher_the_value_the_more_spoken_word_the_song_contains,Popularity-_The_higher_the_value_the_more_popular_the_song_is,Predicted Category NB,Predicted Category NN,Predicted Category Clustering KM,Jaccard Similarity
428,Make Me Like You,Gwen Stefani,dance pop,2016,118,71,74,-5,13,80,217,4,5,50,[1],[[5.060336633858113e-10]],[3],0.076324
23,Misery,Maroon 5,pop,2010,103,81,70,-5,22,73,216,0,4,65,[1],[[6.590556243258183e-09]],[3],0.075647
543,Sign of the Times,Harry Styles,dance pop,2018,120,57,52,-5,11,21,341,3,3,72,[1],[[5.510726057584558e-10]],[3],0.074412
28,Teenage Dream,Katy Perry,dance pop,2010,120,80,72,-5,13,59,228,2,4,63,[1],[[4.536042971845813e-10]],[3],0.066000
388,Cold Water (feat. Justin Bieber & M?,Major Lazer,dance pop,2016,93,80,61,-5,16,50,185,7,4,73,[1],[[5.077139385457148e-08]],[3],0.063824


In [14]:
#For the sake of possibly not anwsering all of task 3, here are all the requierments:
print("User has been categorized into the " + str(chosen_category) + " category based on the NB prediction")
print("------------------------------------------------")
print("The suggested songs are : ")
print(dfs.head(5)['title'])
print("------------------------------------------------")
print("Happy Listening!! ")



User has been categorized into the ['1'] category based on the NB prediction
------------------------------------------------
The suggested songs are : 
428                        Make Me Like You
23                                   Misery
543                       Sign of the Times
28                            Teenage Dream
388    Cold Water (feat. Justin Bieber & M?
Name: title, dtype: object
------------------------------------------------
Happy Listening!! 
